In [ ]:
!python -m pip install --upgrade pip

!pip install numpy
!pip install scikit-learn
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install opencv-python
!pip install tensorflow
!pip install Pillow
!pip install xml-python
!pip install glob2

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2
import seaborn as sns
import PIL
import xml.etree.ElementTree as ET
import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('D:/DataScience/FaceMaskDetection/Input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Path
img_dir = 'D:/DataScience/FaceMaskDetection/Input/images'
annotation_dir = 'D:/College/DataScience/FaceMaskDetection/Input/annotations'

input_dir = 'D:/DataScience/FaceMaskDetection/Input'
output_dir = 'D:/DataScience/FaceMaskDetection/Output'

In [ ]:
# sample visuaization
for idx, image in enumerate(os.listdir(img_dir)):
    img = cv2.imread(os.path.join(img_dir, image), 1)
    plt.imshow(img)
    plt.show()
    
    if idx == 3:
        break

In [ ]:
# make img_file_path
img_file_path = []
for img in os.listdir(img_dir):
    image  = cv2.imread(os.path.join(img_dir, img), 0)
    img_file_path.append(f'{img}')

In [ ]:
df = { 'name': [],
        'label': [],
      'width': [],
      'height': [],
     'xmin': [],
     'ymin': [],
     'xmax': [],
     'ymax': []}

for idx, anno in enumerate(glob.glob(annotation_dir + '/*.xml')):
    trees = ET.parse(anno)
    
    #print(anno)
    root = trees.getroot()
    width, height = [], []
    for item in root.iter():
        #print(item)
               
        if item.tag == 'size':
            for attr in list(item):
                if attr.tag == 'width':
                    width = int(round(float(attr.text)))
                if attr.tag == 'height':
                    height = int(round(float(attr.text)))
                    
        if item.tag == 'object':
            for attr in list(item):
                if 'name' in attr.tag:
                    label = attr.text
                    df['label'] += [label]
                    df['width'] += [width]
                    df['height'] += [height]
                    #dataset['name']+=[anno.split('/')[-1][0:-4]] 
                    df['name'] += [anno.split('/')[-1][0:-4]]
                    
                if 'bndbox' in attr.tag:
                    for dim in attr:
                        if dim.tag == 'xmin':
                            xmin = int(round(float(dim.text)))
                            df['xmin'] += [xmin]
                            
                        if dim.tag == 'ymin':
                            ymin = int(round(float(dim.text)))
                            df['ymin'] += [ymin]
                        if dim.tag == 'xmax':
                            xmax = int(round(float(dim.text)))
                            df['xmax'] += [xmax]
                        if dim.tag == 'ymax':
                            ymax = int(round(float(dim.text)))
                            df['ymax'] += [ymax]

In [ ]:
df1 = pd.DataFrame(df)
df1.head()

In [ ]:
df1.info()

In [ ]:
label_map = { 'without_mask': 0,
            'with_mask': 1,
            'mask_weared_incorrect': 2}

df1['class'] = df1['label'].map(label_map)

In [ ]:
# split train, test, val data
from sklearn.model_selection import train_test_split

train, test = train_test_split(img_file_path, test_size=0.2, random_state=101)
train, val = train_test_split(train, test_size=0.15, random_state=101)

In [ ]:
# yolo v5
%cd Output
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -qr requirements.txt

In [ ]:
# create a separate folder structure for train, validation and test image and validation files
os.chdir(r'D:/DataScience/FaceMaskDetection/Output')
os.mkdir(r'D:/DataScience/FaceMaskDetection/Output/yolov5/data/train')
os.mkdir(r'D:/DataScience/FaceMaskDetection/Output/yolov5/data/val')
os.mkdir(r'D:/DataScience/FaceMaskDetection/Output/yolov5/data/test')
os.mkdir(r'D:/DataScience/FaceMaskDetection/Output/yolov5/data/train/images')
os.mkdir(r'D:/DataScience/FaceMaskDetection/Output/yolov5/data/train/labels')
os.mkdir(r'D:/DataScience/FaceMaskDetection/Output/yolov5/data/val/images')
os.mkdir(r'D:/DataScience/FaceMaskDetection/Output/yolov5/data/val/labels')
os.mkdir(r'D:/DataScience/FaceMaskDetection/Output/yolov5/data/test/images')
os.mkdir(r'D:/DataScience/FaceMaskDetection/Output/yolov5/data/test/labels')

In [ ]:
# copy the image data in the yolov5 folder 
def copy_image_file(image_items, folder_name):
    
    for image in image_items:
        img = PIL.Image.open(f'{img_dir}/{image}')
        img1 = img.resize((640, 480))
        _ = img1.save(f'{output_dir}/yolov5/data/{folder_name}/images/{image}')

copy_image_file(train, 'train')
copy_image_file(val, 'val')
copy_image_file(test, 'test')

In [ ]:
df1['xmin'] = (640/df1['width']) * df1['xmin']
df1['ymin'] = (480/df1['height']) * df1['ymin']
df1['xmax'] = (640/df1['width']) * df1['xmax']
df1['ymax'] = (480/df1['height']) * df1['ymax']
df1.head()

In [ ]:
df1[['xmin', 'ymin', 'xmax', 'ymax']] = df1[['xmin', 'ymin', 'xmax', 'ymax']].astype('int')

In [ ]:
WIDTH = 640
HEIGHT = 480

In [ ]:
df1['x_center'] = (df1['xmin']+df1['xmax'])/(2*WIDTH)
df1['y_center'] = (df1['ymin']+df1['ymax'])/(2*HEIGHT)
df1['box_width'] = (df1['xmax']-df1['xmin'])/ WIDTH
df1['box_height'] = (df1['ymax']-df1['ymin'])/ HEIGHT

In [ ]:
df1.head()

In [ ]:
df1 = df1.astype(str)

In [ ]:
img_file_path[:5]

In [ ]:
# copy the label data in the yolov5 folder 

def copy_label(label_items, folder_name):
    file_name = [x.split('.')[0] for x in img_file_path]
    for name in file_name:
        data = df1[df1.name == name]

        box_list = []
        for idx in range(len(data)):
            row = data.iloc[idx]
            box_list.append(row['class']+" "+row['x_center']+" "+row['y_center']+" "+ row['box_width']+" "+row['box_height'])

        text = "\n".join(box_list)
        with open(f'{output_dir}/yolov5/data/{folder_name}/labels/{name}.txt', 'w') as file:
            file.write(text)

In [ ]:
copy_label(train, 'train')
copy_label(val, 'val')
copy_label(test, 'test')

In [ ]:
os.chdir(f'D:/DataScience/FaceMaskDetection/Output/yolov5/data/train/labels')

In [ ]:
# Configure .yaml file 
yaml_file = """train: D:/DataScience/FaceMaskDetection/Output/yolov5/data/train/images
val: D:/DataScience/FaceMaskDetection/Output/yolov5/data/val/images
                
nc: 3

names: [without_mask, with_mask, mask_weared_incorrect]"""

with open('D:/DataScience/FaceMaskDetection/Output/yolov5/data/data.yaml', 'w') as f:
    f.write(yaml_file)

## Model Training

In [ ]:
!pip uninstall comet-ml -y

In [ ]:
!python D:/DataScience/FaceMaskDetection/Output/yolov5/train.py --img 640 --epochs 50 --batch 32 --data D:/DataScience/FaceMaskDetection/Output/yolov5/data/data.yaml --cfg D:/DataScience/FaceMaskDetection/Output/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name mask_wearing_yolov5